Count number of classes 

In [1]:
import pandas as pd

# Assuming your CSV file has columns 'id' and 'label'
csv_file_path = 'train.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Count the number of images for each label
label_counts = df['label'].value_counts()

# Count the number of unique labels
unique_labels_count = len(label_counts)

# Calculate the average number of images per label
average_images_per_label = label_counts.mean()

# Display the results
print(f"Number of unique labels: {unique_labels_count}")
print("\nNumber of images for each label:")
print(label_counts)
print(f"\nAverage number of images per label: {average_images_per_label:.2f}")


Number of unique labels: 100

Number of images for each label:
68    49
45    47
32    46
84    46
9     46
      ..
89    34
15    34
34    33
53    30
88    28
Name: label, Length: 100, dtype: int64

Average number of images per label: 41.35


Custom Dataset Builder

In [1]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx, 0]
        img_path = f'{img_name}'
        image = Image.open(img_path)

        label = int(self.data_frame.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

# Example usage:
transform = transforms.Compose([
    #transforms.Resize((299, 299)),  # Assuming Inception input size
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CustomDataset(csv_file='train.csv', root_dir='', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


Inception Model Download

In [2]:
from torchvision import models
import torch.optim as optim
import torch.nn as nn
import torch

# Load the pre-trained Inception model
inception_model = models.inception_v3(pretrained=True)

# Modify the last classification layer
in_features = inception_model.fc.in_features
inception_model.fc = nn.Linear(in_features, 100)  # Assuming 100 classes

# You might want to move your model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception_model = inception_model.to(device)


C:\Users\seher\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\seher\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception Model

In [3]:
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
import torch.autograd.profiler as profiler

# Choose your learning rate
learning_rate = 0.001

# Choose the number of epochs
num_epochs = 20

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(inception_model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# Function to calculate accuracy and precision
def calculate_metrics(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    accuracy = correct / labels.size(0)

    return accuracy

for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    epoch_precision = 0.0

    # Enable profiler for the entire epoch
    with profiler.profile(record_shapes=True) as prof:
        with profiler.record_function("train_epoch"):
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs, aux_outputs = inception_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                scheduler.step()

                # Update metrics
                epoch_loss += loss.item()
                epoch_accuracy += calculate_metrics(outputs, labels)

    # Print and save metrics after each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Learning Rate: {scheduler.get_last_lr()}')
    print(f'  Loss: {epoch_loss / len(train_loader)}')
    print(f'  Accuracy: {epoch_accuracy / len(train_loader)}')

    # Save model parameters
    torch.save(inception_model.state_dict(), f'model_epoch_{epoch + 1}.pth')

# Use the profiler results
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


Resnet Model

In [5]:
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Load pre-trained ResNet model
resnet = models.resnet18(pretrained=True)

# Modify the final fully connected layer
num_classes = 100
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    resnet.train()

    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = resnet(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

    # Print training loss and accuracy after each epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Accuracy: {correct/total * 100:.2f}%")

    # Save the model after each epoch
    model_save_path = f"resnet_epoch_{epoch + 1}.pth"
    torch.save(resnet.state_dict(), model_save_path)
    print(f"Model saved at: {model_save_path}")

# Optionally, save the final model after training
final_model_path = "resnet_final_model.pth"
torch.save(resnet.state_dict(), final_model_path)
print(f"Final model saved at: {final_model_path}")

C:\Users\seher\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\seher\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 3.6018657244168795, Training Accuracy: 28.21%
Model saved at: resnet_epoch_1.pth
Epoch 2/10, Loss: 1.8032306029246403, Training Accuracy: 51.28%
Model saved at: resnet_epoch_2.pth
Epoch 3/10, Loss: 0.9729193990047161, Training Accuracy: 66.67%
Model saved at: resnet_epoch_3.pth
Epoch 4/10, Loss: 0.5629766395458808, Training Accuracy: 84.62%
Model saved at: resnet_epoch_4.pth
Epoch 5/10, Loss: 0.311437391776305, Training Accuracy: 76.92%
Model saved at: resnet_epoch_5.pth
Epoch 6/10, Loss: 0.22773394802441962, Training Accuracy: 89.74%
Model saved at: resnet_epoch_6.pth
Epoch 7/10, Loss: 0.16603349430056719, Training Accuracy: 89.74%
Model saved at: resnet_epoch_7.pth
Epoch 8/10, Loss: 0.15666354808669825, Training Accuracy: 100.00%
Model saved at: resnet_epoch_8.pth
Epoch 9/10, Loss: 0.10420793420993364, Training Accuracy: 97.44%
Model saved at: resnet_epoch_9.pth
Epoch 10/10, Loss: 0.103273643553257, Training Accuracy: 100.00%
Model saved at: resnet_epoch_10.pth
Fina

Test Dataset

In [13]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os, csv


class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [
            os.path.join(root_dir, img_name) for img_name in os.listdir(root_dir)
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return img_path, image


transform = transforms.Compose(
    [
        # transforms.Resize((299, 299)),  # Assuming Inception input size
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)



Inference

In [14]:
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import os, csv

# Define the number of classes
num_classes = 100
fourth_epoch_path="resnet_epoch_4.pth"
# Load the saved state_dict
saved_state_dict = torch.load(final_model_path)

# Create a new ResNet model for testing with the correct number of classes
resnet_test = models.resnet18(pretrained=False)
resnet_test.fc = nn.Linear(resnet_test.fc.in_features, num_classes)

# Load the state_dict into the new testing model
resnet_test.load_state_dict(saved_state_dict)

# Set the model to evaluation mode
resnet_test.eval()
print("starting eval")
# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet_test.to(device)

# Define the transform for testing images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the test dataset and data loader
test_dataset = TestDataset(root_dir="test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predictions = []
print("performing inference")
# Perform inference
with torch.no_grad():
    for img_path, inputs in test_loader:
        inputs = inputs.to(device)

        # Forward pass
        outputs = resnet_test(inputs)

        # Get predicted labels
        _, predicted = torch.max(outputs, 1)

        # Store predictions
        for path, label in zip(img_path, predicted.cpu().numpy()):
            predictions.append({"id": path.replace("\\","/"), "Predicted": label})

# Save predictions to a CSV file
csv_file = "predictions.csv"
with open(csv_file, "w", newline="") as csvfile:
    fieldnames = ["id", "Predicted"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for prediction in predictions:
        writer.writerow(prediction)

print(f"Predictions saved to {csv_file}")


starting eval
performing inference
Predictions saved to predictions.csv
